## Using Generative AI to create a sketchnote of an article

This notebook uses GenAI to extract key points from an article and then connect them into a diagram.

We do initial exploration with prompts, then look at a way to pull it all together into an agent framework for maintainability.

In [1]:
#%pip install --quiet --upgrade -r requirements.txt

In [1]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv("../genai_agents/keys.env")
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [6]:
## params
params = {
    "article_url": "https://en.wikipedia.org/wiki/Diwali",
    "num_points": 10,
    "topic": "Festivals of the World"
}

## Get text of article

In [3]:
import pdfkit
pdfkit.from_url(params['article_url'], "article.pdf")

True

In [4]:
pdf_file = genai.upload_file("article.pdf")

In [5]:
print(pdf_file)

genai.File({
    'name': 'files/hxrevzklojbz',
    'display_name': 'article.pdf',
    'mime_type': 'application/pdf',
    'sha256_hash': 'NmFhYWUyYzMyZDE4NDdlMzZlYzAwY2E3ZmI0ZWQyMjE3MDE1OWE3NmM3ZGQ4YjVmOWMxMjM1YWEzNjM3OWQ4Ng==',
    'size_bytes': '1124899',
    'state': 'ACTIVE',
    'uri': 'https://generativelanguage.googleapis.com/v1beta/files/hxrevzklojbz',
    'create_time': '2024-11-01T22:36:46.713546Z',
    'expiration_time': '2024-11-03T22:36:46.651229138Z',
    'update_time': '2024-11-01T22:36:46.713546Z'})


## Extract keypoints from article.

Try it with a single prompt.

In [7]:
full_prompt = f"""
You are an artist who is creating a sketchnote of an article. You are an expert on {params['topic']}.
The purpose of the sketchnote is to use graphical elements to represent the contents of the article in an engaging and accurate way
to people learning about the topic.

You can use the following elements:
  * Shapes (circle, rectangle, bubble, text)
  * Containers (shaded box that contains multiple elements)
  * Connector (line or arrow that connects two elements, with text that describes the relationship)
  * Illustration (description of illustration to be drawn)

You will follow the following process:
  * Identify the most important {params['num_points']} ideas in the article. These could be direct quotes or summaries but should be one sentence or a sentence fragment.
  * Represent each idea by a shape and assign it a sequential id.
  * Organize the ideas into larger groups. Each group will be represented by a Container. Every container will have a short title of 5 words or less.
  * Identify the connections between shapes and containers. Every connection will be explained by a short phrase of 5 words or less.
  * For each shape, group, and connection, write a detailed description of an illustration that can represent the idea, suitable to use as a prompt to an image generation model.
"""

In [10]:
from pydantic import BaseModel
from enum import IntEnum
from typing import List

class ShapeEnum(str, Enum):
    circle = 'circle'         # represents verbs
    rectange = 'rectangle'    # represents nouns
    bubble = 'bubble'         # represents thoughts or ideas
    text = 'text'             # catch all

class Shape(BaseModel):
    shape_id: str
    shape_type: ShapeEnum
    text: str
    illustration: str
    
class Group(BaseModel):
    shapes: List[str] # list of shape_id
    title: str
    illustration: str
    
class Connection(BaseModel):
    from_shape: str # shape_id
    to_shape: str # shape_id
    relationship: str
    illustration: str

class Sketchnote(BaseModel):
    title: str
    ideas: List[Shape]
    groups: List[Group]
    connections: List[Connection]

In [14]:
import json

model = genai.GenerativeModel(
    "gemini-1.5-flash-001",
    system_instruction=[full_prompt]
)
generation_config={
    "temperature": 0.7,
    "max_output_tokens": params['num_points']*10000,
    "response_mime_type": "application/json",
    "response_schema": Sketchnote
}
iter = 1
while iter < 10:
    print(f"Generating content ... Attempt {iter}")
    responses = model.generate_content(
        [pdf_file],
        generation_config=generation_config,
        stream=False
    )
    iter = iter + 1
    if (str(responses.candidates[0].finish_reason) == "FinishReason.STOP"):
        # complete JSON?
        try:
            sketchnote = json.loads(responses.text)
            print("Success")
            break
        except:
            print("Error! Got incomplete JSON")
    else:
        print(f"Error! Got finish reason of {str(responses.candidates[0].finish_reason)}")

Generating content ... Attempt 1
Error! Got incomplete JSON
Generating content ... Attempt 2
Error! Got incomplete JSON
Generating content ... Attempt 3
Error! Got incomplete JSON
Generating content ... Attempt 4
Success


In [15]:
print(len(responses.text))

2421


In [16]:
len(sketchnote['ideas'])

13

In [19]:
import json
with open('sketchnote.json', 'w') as ofp:
    ofp.write(json.dumps(sketchnote))

In [17]:
sketchnote

{'connections': [{'from_shape': 'S1',
   'relationship': 'Is celebrated',
   'to_shape': 'G1'},
  {'from_shape': 'S2', 'relationship': 'Is celebrated', 'to_shape': 'G1'},
  {'from_shape': 'S3', 'relationship': 'Is celebrated', 'to_shape': 'G1'},
  {'from_shape': 'S4', 'relationship': 'Is celebrated', 'to_shape': 'G1'},
  {'from_shape': 'S5', 'relationship': 'Is celebrated', 'to_shape': 'G1'},
  {'from_shape': 'S6', 'relationship': 'Is celebrated', 'to_shape': 'G1'},
  {'from_shape': 'S7', 'relationship': 'Is celebrated', 'to_shape': 'G1'},
  {'from_shape': 'S8', 'relationship': 'Is celebrated', 'to_shape': 'G1'},
  {'from_shape': 'S9', 'relationship': 'Is celebrated', 'to_shape': 'G1'},
  {'from_shape': 'S10', 'relationship': 'Is celebrated', 'to_shape': 'G1'},
  {'from_shape': 'G1', 'relationship': 'Is celebrated', 'to_shape': 'G2'},
  {'from_shape': 'G2', 'relationship': 'Is celebrated', 'to_shape': 'G3'}],
 'groups': [{'shapes': ['S1',
    'S2',
    'S3',
    'S4',
    'S5',
    'S6

## Convert json to Graph

In [20]:
import json
with open("sketchnote.json", "r") as ifp:
    sketchnote = json.load(ifp)

In [ ]:
with open("sketchnote.dot", "w") as ofp:
    ofp.write("digraph {")
    for group in sketchnote['groups']:
        ofp.write(f"subgraph {group[''}")